In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import pandas_profiling as pd_prof
import missingno as misno
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
from subprocess import check_output
print(check_output(['ls','input']).decode('utf8'))

/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


data_dict.xlsx
sample_submission_fBo3EW5.csv
test_9tJUnaB.zip
test.csv
train.csv
train_zsTMYVA.zip



In [2]:
train = pd.read_csv('input/train.csv', low_memory=False)
test = pd.read_csv('input/test.csv', low_memory=False)

In [3]:
print(train.shape, test.shape)


(300000, 377) (200000, 376)


In [4]:
le = LabelEncoder()
data_sets = [train,test]
for data in data_sets:
    for col in [x for x in data.columns if x not in ['UCIC_ID','Responders']]:
        if data[col].dtypes == 'object':
            data[col] = data[col].fillna('none')
            data[col] = le.fit_transform(data[col])
            if data[col].max() < 255:
                data[col] = data[col].astype(np.int8)
            else:
                data[col] = data[col].astype(np.int32)
                
        if data[col].dtypes == 'int64':        
            data[col] = data[col].fillna(-99).astype(np.int32)
            
        if data[col].dtypes == 'float64':        
            data[col] = data[col].fillna(-99.0).astype(np.float32)

In [5]:
constant_cols = []
for col in [x for x in train.columns if x not in ['UCIC_ID','Responders']]:
    if len(train[col].value_counts()) == 1:
        constant_cols.append(col)

In [6]:
for data in data_sets:
    data.drop(constant_cols, inplace=True, axis=1)

In [7]:
#test.head(5)

In [8]:
responders= train['Responders'].values
train.drop(['UCIC_ID','Responders'], inplace=True, axis=1)
ucic_id = test['UCIC_ID'].values
test.drop(['UCIC_ID'], inplace=True, axis=1)

del data_sets

In [28]:
params = {'learning_rate': 0.02, 
          'num_leaves':78,
          'min_data_in_leaf': 350, #130
          'max_depth': 6, 
          'colsample_bytree': 0.522,
          'boosting': 'gbdt', 
          'objective': 'binary', 
          'metric': 'auc',           
          'seed': 32}

In [29]:
lgb_train = lgb.Dataset(train, label=responders)

In [30]:
lgb_cv = lgb.cv(params, lgb_train, num_boost_round=3000, nfold= 4, shuffle=True, 
                stratified=True, verbose_eval=20, early_stopping_rounds=40)

[20]	cv_agg's auc: 0.842172 + 0.00262201
[40]	cv_agg's auc: 0.84442 + 0.00270966
[60]	cv_agg's auc: 0.847431 + 0.00251467
[80]	cv_agg's auc: 0.849422 + 0.00236684
[100]	cv_agg's auc: 0.851445 + 0.00248168
[120]	cv_agg's auc: 0.853149 + 0.00271913
[140]	cv_agg's auc: 0.855154 + 0.00246781
[160]	cv_agg's auc: 0.857767 + 0.00228698
[180]	cv_agg's auc: 0.859703 + 0.0022173
[200]	cv_agg's auc: 0.86077 + 0.0022275
[220]	cv_agg's auc: 0.862089 + 0.00210663
[240]	cv_agg's auc: 0.863123 + 0.00201232
[260]	cv_agg's auc: 0.863934 + 0.00196419
[280]	cv_agg's auc: 0.864587 + 0.00189363
[300]	cv_agg's auc: 0.865101 + 0.00186393
[320]	cv_agg's auc: 0.865544 + 0.00185025
[340]	cv_agg's auc: 0.865919 + 0.0018024
[360]	cv_agg's auc: 0.866265 + 0.00176355
[380]	cv_agg's auc: 0.866595 + 0.00173117
[400]	cv_agg's auc: 0.866884 + 0.00173581
[420]	cv_agg's auc: 0.867152 + 0.00173894
[440]	cv_agg's auc: 0.867378 + 0.00169104
[460]	cv_agg's auc: 0.867559 + 0.00165799
[480]	cv_agg's auc: 0.867773 + 0.00163923
[

In [31]:
nround = lgb_cv['auc-mean'].index(np.max(lgb_cv['auc-mean']))
#print(nround)

In [32]:
model_0 = lgb.train(params, lgb_train, num_boost_round=nround)

In [33]:
pred_0 = model_0.predict(test)

In [34]:
future_df = pd.DataFrame({'futures':model_0.feature_name(),'score':model_0.feature_importance()})
col_to_use = future_df[future_df['score'] > 50]['futures'].tolist()


In [35]:
lgb_train = lgb.Dataset(train[col_to_use], label=responders)

In [36]:
lgb_cv = lgb.cv(params, lgb_train, num_boost_round=3000, nfold= 4, shuffle=True, 
                stratified=True, verbose_eval=20, early_stopping_rounds=40)

[20]	cv_agg's auc: 0.840249 + 0.00285866
[40]	cv_agg's auc: 0.844132 + 0.00281133
[60]	cv_agg's auc: 0.846308 + 0.00278241
[80]	cv_agg's auc: 0.848654 + 0.00245756
[100]	cv_agg's auc: 0.850719 + 0.0024977
[120]	cv_agg's auc: 0.853075 + 0.00254916
[140]	cv_agg's auc: 0.854882 + 0.00249501
[160]	cv_agg's auc: 0.856401 + 0.00244452
[180]	cv_agg's auc: 0.858619 + 0.00228509
[200]	cv_agg's auc: 0.860184 + 0.00217217
[220]	cv_agg's auc: 0.861677 + 0.00214147
[240]	cv_agg's auc: 0.862747 + 0.00209752
[260]	cv_agg's auc: 0.863635 + 0.00206858
[280]	cv_agg's auc: 0.864338 + 0.00204912
[300]	cv_agg's auc: 0.864891 + 0.00201582
[320]	cv_agg's auc: 0.865401 + 0.00197399
[340]	cv_agg's auc: 0.86581 + 0.00192605
[360]	cv_agg's auc: 0.866159 + 0.00190783
[380]	cv_agg's auc: 0.866488 + 0.00187883
[400]	cv_agg's auc: 0.866798 + 0.00185308
[420]	cv_agg's auc: 0.867024 + 0.00182971
[440]	cv_agg's auc: 0.86729 + 0.00184141
[460]	cv_agg's auc: 0.867516 + 0.00179184
[480]	cv_agg's auc: 0.867703 + 0.00178049

In [37]:
nround = lgb_cv['auc-mean'].index(np.max(lgb_cv['auc-mean']))

In [38]:
model_1 = lgb.train(params, lgb_train, num_boost_round=nround)

In [39]:
pred_1 = model_1.predict(test[col_to_use])

In [40]:
sub = pd.DataFrame({'UCIC_ID':ucic_id, 'Responders':(pred_0 + pred_1)/2.})
sub = sub[['UCIC_ID','Responders']]
sub.head(5)

,UCIC_ID,Responders
0,337734,0.283057
1,488166,0.025274
2,410785,0.038830
3,389145,0.110002
4,221090,0.163467


In [41]:
sub.to_csv('lgb_v12.csv', index=False)